### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [110]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [111]:
total_schools = school_data["school_name"].size
total_students = school_data_complete["student_name"].size
total_budget = school_data["budget"].sum()
avg_math_score = student_data["math_score"].mean()
avg_reading_score = student_data["reading_score"].mean()
total_math_passed = student_data.loc[student_data["math_score"] >= 70]
total_math_passed = total_math_passed["math_score"].size
total_reading_passed = student_data.loc[student_data["reading_score"] >= 70]
total_reading_passed = total_reading_passed["reading_score"].size
#gets all students that passed both
total_all_passed = student_data.loc[(student_data["reading_score"] >= 70) & (student_data["math_score"] >= 70)]
#gets size
total_all_passed = total_all_passed[["math_score","reading_score"]].size
#divides by 2 since 2 values were passed in size fetch to return total num
total_all_passed = total_all_passed/2

percent_math_pass = (total_math_passed/total_students)*100
percent_reading_pass = (total_reading_passed/total_students)*100
percent_all_pass = (total_all_passed/total_students)*100

summaryDF = pd.DataFrame([{
    "Total Schools": total_schools,
    "Total Students": total_students,
    "Total Budget": total_budget,
    "Average Math Score": avg_math_score,
    "Average Reading Score": avg_reading_score,
    "Percentage Passed Math": percent_math_pass,
    "Percentage Passed Reading": percent_reading_pass,
    "Overall Passed": percent_all_pass
}])
print(summaryDF)

   Total Schools  Total Students  Total Budget  Average Math Score  \
0             15           39170      24649428           78.985371   

   Average Reading Score  Percentage Passed Math  Percentage Passed Reading  \
0               81.87784               74.980853                  85.805463   

   Overall Passed  
0       65.172326  


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [112]:
#group by school
school_group = school_data_complete.groupby("school_name")
students_by_school = school_group["student_name"].count()
school_type = school_group["type"]
school_type = school_type.unique()

school_budget = school_group["budget"].unique()
school_budget_per_student = school_budget/students_by_school

average_math_per_school = school_group["math_score"].mean()
average_reading_per_school = school_group["reading_score"].mean()




bins = [70, 100]

acedemicscores = school_data_complete[["math_score", "reading_score","student_name","school_name"]]
acedemicscores["passed_math"] = pd.cut(acedemicscores["math_score"], bins,  include_lowest=False)
acedemicscores["passed_reading"] = pd.cut(acedemicscores["reading_score"], bins,  include_lowest=False)
acedemicscores = acedemicscores.groupby("school_name")
math_passed_per_school = acedemicscores["passed_math"].count()
reading_passed_per_school = acedemicscores["passed_reading"].count()



# math_passed_per_school = math_passed_per_school.value_counts(bins=bins)

# reading_passed_per_school = reading_passed_per_school.value_counts(bins=bins)

# both_passed_per_school = acedemicscores[["student_name", "passed_math", "passed_reading"]]

both_passed_per_school = acedemicscores[["passed_math", "passed_reading"]].count()
both_passed_per_school = both_passed_per_school/2
both_passed_per_school = both_passed_per_school["passed_math"].add(both_passed_per_school["passed_reading"])

percent_math_passed_per_school = (math_passed_per_school/students_by_school)*100
percent_reading_passed_per_school = (reading_passed_per_school/students_by_school)*100
percent_both_passed_per_school = (both_passed_per_school/students_by_school)*100

percent_math_passed_per_school = percent_math_passed_per_school.rename("math percentage")

percent_reading_passed_per_school = percent_reading_passed_per_school.rename("reading percentage")


summary_by_schoolDF = pd.merge(school_type,school_budget, "outer", left_on="school_name", right_index=True)
summary_by_schoolDF = summary_by_schoolDF.merge(students_by_school.rename("total students"), "outer", right_on="school_name",
                                                left_index=True)
summary_by_schoolDF = summary_by_schoolDF.merge(school_budget_per_student.rename("budget/student"), "outer",
                                                right_on="school_name", left_index=True)
summary_by_schoolDF = summary_by_schoolDF.merge(average_math_per_school.rename("Average Math Score"), "outer", right_on="school_name",
                                                left_index=True)
summary_by_schoolDF = summary_by_schoolDF.merge(average_reading_per_school.rename("Average Reading Score"),
                                                "outer", right_on="school_name", left_index=True)
summary_by_schoolDF = summary_by_schoolDF.merge(percent_math_passed_per_school.rename("% passed math"),
                                                "outer", right_on="school_name", left_index=True)
summary_by_schoolDF = summary_by_schoolDF.merge(percent_reading_passed_per_school.rename("% passed reading"),
                                                "outer", right_on="school_name", left_index=True)
summary_by_schoolDF = summary_by_schoolDF.merge(percent_both_passed_per_school.rename("% passed both"),
                                                "outer", right_on="school_name", left_index=True)
print(summary_by_schoolDF)

                             type     budget  total students budget/student  \
school_name                                                                   
Bailey High School     [District]  [3124928]            4976        [628.0]   
Cabrera High School     [Charter]  [1081356]            1858        [582.0]   
Figueroa High School   [District]  [1884411]            2949        [639.0]   
Ford High School       [District]  [1763916]            2739        [644.0]   
Griffin High School     [Charter]   [917500]            1468        [625.0]   
Hernandez High School  [District]  [3022020]            4635        [652.0]   
Holden High School      [Charter]   [248087]             427        [581.0]   
Huang High School      [District]  [1910635]            2917        [655.0]   
Johnson High School    [District]  [3094650]            4761        [650.0]   
Pena High School        [Charter]   [585858]             962        [609.0]   
Rodriguez High School  [District]  [2547363]        

<ipython-input-112-c498f4b2e18a>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  acedemicscores["passed_math"] = pd.cut(acedemicscores["math_score"], bins,  include_lowest=False)
<ipython-input-112-c498f4b2e18a>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  acedemicscores["passed_reading"] = pd.cut(acedemicscores["reading_score"], bins,  include_lowest=False)


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [113]:
top_performers = percent_both_passed_per_school.sort_values(ascending=False).head(5)

print(top_performers)

school_name
Wilson High School     92.093736
Pena High School       91.943867
Wright High School     91.861111
Cabrera High School    91.711518
Holden High School     91.686183
dtype: float64


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [114]:
worst_performers = percent_both_passed_per_school.sort_values(ascending=True).head(5)


print(worst_performers)

school_name
Rodriguez High School    70.905226
Huang High School        71.066164
Johnson High School      71.067003
Figueroa High School     71.091896
Hernandez High School    71.467098
dtype: float64


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [115]:
grade_data = student_data[["grade", "math_score", "reading_score", "school_name"]]

grade9 = grade_data.loc[student_data["grade"] == "9th"]
grade10 = grade_data.loc[student_data["grade"] == "10th"]
grade11 = grade_data.loc[student_data["grade"] == "11th"]
grade12 = grade_data.loc[student_data["grade"] == "12th"]

group9 = grade9.groupby("school_name")
group10 = grade10.groupby("school_name")
group11 = grade11.groupby("school_name")
group12 = grade12.groupby("school_name")

group9_math = group9["math_score"].mean()
group10_math = group10["math_score"].mean()
group11_math = group11["math_score"].mean()
group12_math = group12["math_score"].mean()

group9_math = group9_math.drop(columns="reading_score")
group10_math = group10_math.drop(columns="reading_score")
group11_math = group11_math.drop(columns="reading_score")
group12_math = group12_math.drop(columns="reading_score")

math_gradedf = pd.merge(group9_math,group10_math, "outer", left_on="school_name",right_index=True)
math_gradedf = math_gradedf.rename(columns={"math_score_x":"9th grade avg math scores" , "math_score_y":"10th grade avg math scores"})
math_gradedf = math_gradedf.merge(group11_math, "outer", left_on="school_name",right_index=True)
math_gradedf = math_gradedf.rename(columns={"math_score":"11th grade avg math scores"})
math_gradedf = math_gradedf.merge(group12_math, "outer", left_on="school_name",right_index=True)
math_gradedf = math_gradedf.rename(columns={"math_score":"12th grade avg math scores" })
print(math_gradedf)

                       9th grade avg math scores  10th grade avg math scores  \
school_name                                                                    
Bailey High School                     77.083676                   76.996772   
Cabrera High School                    83.094697                   83.154506   
Figueroa High School                   76.403037                   76.539974   
Ford High School                       77.361345                   77.672316   
Griffin High School                    82.044010                   84.229064   
Hernandez High School                  77.438495                   77.337408   
Holden High School                     83.787402                   83.429825   
Huang High School                      77.027251                   75.908735   
Johnson High School                    77.187857                   76.691117   
Pena High School                       83.625455                   83.372000   
Rodriguez High School                  7

## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [116]:
group9_reading = group9["reading_score"].mean()
group10_reading = group10["reading_score"].mean()
group11_reading = group11["reading_score"].mean()
group12_reading = group12["reading_score"].mean()

group9_reading = group9_reading.drop(columns="math_score")
group10_reading = group10_reading.drop(columns="math_score")
group11_reading = group11_reading.drop(columns="math_score")
group12_reading = group12_reading.drop(columns="math_score")

read_gradedf = pd.merge(group9_reading,group10_reading, "outer", left_on="school_name",right_index=True)
read_gradedf = read_gradedf.rename(columns={"reading_score_x":"9th grade avg reading scores" , "reading_score_y":"10th grade avg reading scores"})
read_gradedf = read_gradedf.merge(group11_reading, "outer", left_on="school_name",right_index=True)
read_gradedf = read_gradedf.rename(columns={"reading_score":"11th grade avg reading scores"})
read_gradedf = read_gradedf.merge(group12_reading, "outer", left_on="school_name",right_index=True)
read_gradedf = read_gradedf.rename(columns={"reading_score":"12th grade avg reading scores" })
print(read_gradedf)

                       9th grade avg reading scores  \
school_name                                           
Bailey High School                        81.303155   
Cabrera High School                       83.676136   
Figueroa High School                      81.198598   
Ford High School                          80.632653   
Griffin High School                       83.369193   
Hernandez High School                     80.866860   
Holden High School                        83.677165   
Huang High School                         81.290284   
Johnson High School                       81.260714   
Pena High School                          83.807273   
Rodriguez High School                     80.993127   
Shelton High School                       84.122642   
Thomas High School                        83.728850   
Wilson High School                        83.939778   
Wright High School                        83.833333   

                       10th grade avg reading scores  \
school_n

## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [117]:
spending_data = school_data_complete[["budget", "math_score", "reading_score", "school_name", "student_name"]]

#spending_bins = [0 - 600/ 600 - 625/ 625 - 650/ 650 - 675]

spending1 = summary_by_schoolDF.loc[summary_by_schoolDF["budget/student"] <= 600]
spending2 = summary_by_schoolDF.loc[(summary_by_schoolDF["budget/student"] > 600) & (summary_by_schoolDF["budget/student"] <= 625)]
spending3 = summary_by_schoolDF.loc[(summary_by_schoolDF["budget/student"] > 625) & (summary_by_schoolDF["budget/student"] <= 650)]
spending4 = summary_by_schoolDF.loc[(summary_by_schoolDF["budget/student"] > 650) & (summary_by_schoolDF["budget/student"] <= 675)]

spending1 = spending1.drop(columns={"type", "budget", "total students"})
spending2 = spending2.drop(columns={"type", "budget", "total students"})
spending3 = spending3.drop(columns={"type", "budget", "total students"})
spending4 = spending4.drop(columns={"type", "budget", "total students"})

spendingDF = pd.merge(spending1,spending2, "outer", left_on="school_name", right_index=True)
spendingDF = spendingDF.rename(columns={"budget/student_x":"<=$600/student","Average Math Score_x":"Average math:<=$600",
                                        "Average Reading Score_x":"Average reading <=$600", "% passed math_x":"%passed math <=600",
                                        "% passed reading_x":"%passed reading <=600",  "% passed both_x":"%passed both <=600",
                                        "budget/student_y":"$600-625/student", "Average Math Score_y":"$600-625 Avg math score",
                                        "Average Reading Score_y":"$600-625 Avg reading score", "% passed math_y":"%passed math 600-625",
                                        "% passed reading_y":"%passed reading 600-625", "% passed both_y": "%passed both 600-625"})
spendingDF = spendingDF.merge(spending3, "outer", left_on="school_name", right_index=True)
spendingDF = spendingDF.rename(columns={"budget/student":"$625-650/student","Average Math Score":"Average math:$625-650",
                                        "Average Reading Score":"Average reading $625-650", "% passed math":"%passed math 625-650",
                                        "% passed reading":"%passed reading 625-650",  "% passed both":"%passed both 625-650"})
spendingDF = spendingDF.merge(spending4, "outer", left_on="school_name", right_index=True)
spendingDF = spendingDF.rename(columns={"budget/student":"$650-675/student","Average Math Score":"Average math:$650-675",
                                        "Average Reading Score":"Average reading $650-675", "% passed math":"%passed math 650-675",
                                        "% passed reading":"%passed reading 650-675",  "% passed both":"%passed both 650-675"})

print(spendingDF)

                               school_name <=$600/student  \
Cabrera High School    Cabrera High School        [582.0]   
Holden High School      Holden High School        [581.0]   
Shelton High School    Shelton High School        [600.0]   
Wilson High School      Wilson High School        [578.0]   
Wright High School      Wright High School        [583.0]   
NaN                    Griffin High School            NaN   
NaN                       Pena High School            NaN   
NaN                     Bailey High School            NaN   
NaN                   Figueroa High School            NaN   
NaN                       Ford High School            NaN   
NaN                    Johnson High School            NaN   
NaN                  Rodriguez High School            NaN   
NaN                     Thomas High School            NaN   
NaN                  Hernandez High School            NaN   
NaN                      Huang High School            NaN   

                     Av

## Scores by School Size

* Perform the same operations as above, based on school size.

## Scores by School Type

* Perform the same operations as above, based on school type


In [118]:
typedata = school_data_complete[["type", "math_score", "reading_score"]]
charter = typedata.loc[typedata["type"] == "Charter"]
district =typedata.loc[typedata["type"] == "District"]
districtgroup = district.groupby("type")
chartergroup = charter.groupby("type")
district_avg_math = districtgroup["math_score"].mean()
district_avg_read = districtgroup["reading_score"].mean()
charter_avg_math = chartergroup["math_score"].mean()
charter_avg_reading = chartergroup["reading_score"].mean()

charter_reading_size = chartergroup["reading_score"].count()
district_reading_size = districtgroup["reading_score"].count()
charter_math_size = chartergroup["math_score"].count()
district_math_size = districtgroup["math_score"].count()

chartertype_reading_passed = typedata.loc[(typedata["reading_score"] >= 70) & (typedata["type"] == "Charter")]

chartertype_reading_passed = chartertype_reading_passed.drop("math_score", axis=1).drop("type",axis=1).size
districtypr_reading_passed = typedata.loc[(typedata["reading_score"] >= 70) & (typedata["type"] == "District")]
districtypr_reading_passed = districtypr_reading_passed.drop("math_score", axis=1).drop("type",axis=1).size

charter_reading_passed = (chartertype_reading_passed/charter_reading_size)*100
district_reading_passed = (districtypr_reading_passed/district_reading_size)*100

districttype_math = typedata.loc[(typedata["math_score"] >= 70) & (typedata["type"] == "District")]
districttype_math= districttype_math.drop_duplicates().drop("reading_score", axis=1).drop("type",axis=1).size

Chartertype_math = typedata.loc[(typedata["math_score"] >= 70) & (typedata["type"] == "Charter")]
Chartertype_math = Chartertype_math.drop_duplicates().drop("reading_score",axis=1).drop("type",axis=1).size
charter_math_passed = (Chartertype_math/charter_math_size)*100
district_math_passed = (districttype_math/district_math_size)*100

chartertype_all_passed = typedata.loc[(typedata["reading_score"] >= 70) & (typedata["math_score"] >= 70) & (typedata["type"] == "Charter")]
districttype_all_passed = typedata.loc[(typedata["reading_score"] >= 70) & (typedata["math_score"] >= 70) & (typedata["type"] == "District")]
chartertype_all_passed = chartertype_all_passed.drop("type",axis=1).drop("reading_score",axis=1).size
districttype_all_passed = districttype_all_passed.drop("type",axis=1).drop("reading_score",axis=1).size
chartertype_all_passed_percent = (chartertype_all_passed/charter_reading_size)*100
districttype_all_passed_percent = (districttype_all_passed/district_reading_size)*100


typeDF = pd.merge(district_avg_read, district_avg_math, "outer", left_on="type", right_index=True)

typeDF = typeDF.merge(district_reading_passed.rename("reading percent"), "outer" ,on="type")
typeDF = typeDF.merge(district_math_passed.rename("math percent"), "outer", on="type")
typeDF = typeDF.merge(districttype_all_passed_percent.rename("Overall percent"),"outer",on="type")


typeDF1 = pd.merge(charter_avg_reading, charter_avg_math, "outer", left_on="type", right_index=True)

typeDF1 = typeDF1.merge(charter_reading_passed.rename("reading percent"), "outer" ,on="type")
typeDF1 = typeDF1.merge(charter_math_passed.rename("math percent"), "outer", on="type")
typeDF1 = typeDF1.merge(chartertype_all_passed_percent.rename("Overall percent"),"outer",on="type")

typeDF2 = pd.concat([typeDF,typeDF1], axis=1)

# typeDF = typeDF.merge(charter_reading_passed, "outer")
# typeDF = typeDF.merge(charter_math_passed, "outer")
# typeDF = typeDF.merge(chartertype_all_passed_percent, "outer")
print(typeDF2)


          reading_score  math_score  reading percent  math percent  \
District      80.962485   76.987026        80.905249      4.114769   
Charter             NaN         NaN              NaN           NaN   

          Overall percent  reading_score  math_score  reading percent  \
District        53.695878            NaN         NaN              NaN   
Charter               NaN      83.902821   83.406183        96.645891   

          math percent  Overall percent  
District           NaN              NaN  
Charter       7.626702        90.560932  


In [119]:


#